# Function Signatures

In [1]:
import inspect
from inspect import signature
from collections import Counter


## Introspecting callables with the Signature object

The `Signature` object represents the call signature of a callable object and its return annotation. To retrieve a Signature object, use the `signature()` function.


`inspect.signature(callable, *, follow_wrapped=True, globals=None, locals=None, eval_str=False)`
> Return a Signature object for the given callable:
> 
> ```python
> from inspect import signature
> def foo(a, *, b:int, **kwargs):
>     pass
> 
> sig = signature(foo)
> str(sig)
> ```
> > '(a, *, b: int, **kwargs)'
> 
> `str(sig.parameters['b'])`
> > 'b: int'
> 
> `sig.parameters['b'].annotation`
> > \<class 'int'>
>
> Accepts a wide range of Python callables, from plain functions and classes to `functools.partial()` objects.
>
> If the passed object has a `__signature__` attribute, this function returns it without further computations.
>
> For objects defined in modules using stringized annotations (`from __future__ import annotations`), `signature()` will attempt to automatically un-stringize the annotations using `get_annotations()`. The _globals_, _locals_, and _eval_str_ parameters are passed into `get_annotations()` when resolving the annotations; see the documentation for `get_annotations()` for instructions on how to use these parameters.
>
> Raises `ValueError` if no signature can be provided, and `TypeError` if that type of object is not supported. Also, if the annotations are stringized, and _eval_str_ is not false, the `eval()` call(s) to un-stringize the annotations in `get_annotations()` could potentially raise any kind of exception.
>
> A slash(/) in the signature of a function denotes that the parameters prior to it are >positional-only. For more info, see the _FAQ entry on positional-only parameters_.
>
> **Note:** Some callables may not be introspectable in certain implementations of Python. For example, in CPython, some built-in functions defined in C provide no metadata about their arguments.


`class inspect.Signature(parameters=None, *, return_annotation=Signature.empty)`
> A Signature object represents the call signature of a function and its return annotation. For each parameter accepted by the function it stores a Parameter object in its parameters collection.
>
> The optional parameters argument is a sequence of Parameter objects, which is validated to check that there are no parameters with duplicate names, and that the parameters are in the right order, i.e. positional-only first, then positional-or-keyword, and that parameters with defaults follow parameters without defaults.
>
> The optional return_annotation argument can be an arbitrary Python object. It represents the “return” annotation of the callable.
>
> Signature objects are immutable. Use `Signature.replace()` to make a modified copy.

**empty**
> A special class-level marker to specify absence of a return annotation.

**parameters**
> An ordered mapping of parameters’ names to the corresponding Parameter objects. Parameters appear in strict definition order, including keyword-only parameters.

**return_annotation**
> The “return” annotation for the callable. If the callable has no “return” annotation, this attribute is set to Signature.empty.

`bind(*args, **kwargs)`
> Create a mapping from positional and keyword arguments to parameters. Returns BoundArguments if *args and **kwargs match the signature, or raises a TypeError.

`bind_partial(*args, **kwargs)`
> Works the same way as Signature.bind(), but allows the omission of some required arguments (mimics functools.partial() behavior.) Returns BoundArguments, or raises a TypeError if the passed arguments do not match the signature.

`replace(*[, parameters][, return_annotation])`
> Create a new Signature instance based on the instance `replace()` was invoked on. It is possible to pass different parameters and/or return_annotation to override the corresponding properties of the base signature. To remove return_annotation from the copied Signature, pass in `Signature.empty`.
>
>```python
>def test(a, b):
>    pass
>
>sig = signature(test)
>new_sig = sig.replace(return_annotation="new return anno")
>str(new_sig)
>```
>> "(a, b) -> 'new return anno'"

`classmethod from_callable(obj, *, follow_wrapped=True, globals=None, locals=None, eval_str=False)`
> Return a Signature (or its subclass) object for a given callable obj.
>
> This method simplifies subclassing of `Signature`:
>
>```python
>class MySignature(Signature):
>    pass
>sig = MySignature.from_callable(sum)
>assert isinstance(sig, MySignature)
```

Its behavior is otherwise identical to that of `signature()`.


`class inspect.Parameter(name, kind, *, default=Parameter.empty, annotation=Parameter.empty)`
> Parameter objects are immutable. Instead of modifying a Parameter object, you can use Parameter.replace() to create a modified copy.

**empty**
> A special class-level marker to specify absence of default values and annotations.

**name**
> The name of the parameter as a string. The name must be a valid Python identifier.


**default**
> The default value for the parameter. If the parameter has no default value, this attribute is set to Parameter.empty.

***annotation***
> The annotation for the parameter. If the parameter has no annotation, this attribute is set to Parameter.empty.

**kind**
> Describes how argument values are bound to the parameter. The possible values are accessible via Parameter (like Parameter.KEYWORD_ONLY), and support comparison and ordering, in the following order:

|Name|Description|Meaning|
|----|-----------|-------|
POSITIONAL_ONLY|'positional-only'|Value must be supplied as a positional argument. Positional only parameters are those which appear before a / entry (if present) in a Python function definition.|
|POSITIONAL_OR_KEYWORD|'positional or keyword'|Value may be supplied as either a keyword or positional argument (this is the standard binding behaviour for functions implemented in Python.)|
|VAR_POSITIONAL|'variadic positional'|A tuple of positional arguments that aren’t bound to any other parameter. This corresponds to a *args parameter in a Python function definition.|
|KEYWORD_ONLY|'keyword-only'|Value must be supplied as a keyword argument. Keyword only parameters are those which appear after a * or *args entry in a Python function definition.|
|VAR_KEYWORD|'variadic keyword'|A dict of keyword arguments that aren’t bound to any other parameter. This corresponds to a **kwargs parameter in a Python function definition.|


> Example: print all keyword-only arguments without default values:
>
> ```python
> def foo(a, b, *, c, d=10):
>     pass
> 
> sig = signature(foo)
> for param in sig.parameters.values():
>     if (param.kind == param.KEYWORD_ONLY and
>                        param.default is param.empty):
>         print('Parameter:', param)
> ```
> >Parameter: c

**kind.description**
> Describes a enum value of Parameter.kind.
>
> Example: print all descriptions of arguments:
>
>```python
> def foo(a, b, *, c, d=10):
>     pass
> 
> sig = signature(foo)
> for param in sig.parameters.values():
>     print(param.kind.description)
> ```
> > positional or keyword
> > positional or keyword
> > keyword-only
> > keyword-only
> 
`replace(*[, name][, kind][, default][, annotation])`
> Create a new Parameter instance based on the instance replaced was invoked on. To override a Parameter attribute, pass the corresponding argument. To remove a default value or/and an annotation from a Parameter, pass Parameter.empty.

> ```python
> from inspect import Parameter
> param = Parameter('foo', Parameter.KEYWORD_ONLY, default=42)
> str(param)
> ```
> > 'foo=42'
> 
> `str(param.replace()) # Will create a shallow copy of 'param'`
> > 'foo=42'
> 
> `str(param.replace(default=Parameter.empty, annotation='spam'))`
> > "foo: 'spam'"


`class inspect.BoundArguments`
> Result of a Signature.bind() or Signature.bind_partial() call. Holds the mapping of arguments to the function’s parameters.

**arguments**
> A mutable mapping of parameters’ names to arguments’ values. Contains only explicitly bound arguments. Changes in arguments will reflect in args and kwargs.
> 
> Should be used in conjunction with Signature.parameters for any argument processing purposes.
> 
> **Note** Arguments for which Signature.bind() or Signature.bind_partial() relied on a default value are skipped. However, if needed, use BoundArguments.apply_defaults() to add them.

**args**
> A tuple of positional arguments values. Dynamically computed from the arguments attribute.

**kwargs**
> A dict of keyword arguments values. Dynamically computed from the arguments attribute.

**signature**
> A reference to the parent Signature object.

`apply_defaults()`
> Set default values for missing arguments.
> 
> For variable-positional arguments (*args) the default is an empty tuple.
> 
> For variable-keyword arguments (**kwargs) the default is an empty dict.
>
> ```python
> def foo(a, b='ham', *args): pass
> ba = inspect.signature(foo).bind('spam')
> ba.apply_defaults()
> ba.arguments
> ```
> . {'a': 'spam', 'b': 'ham', 'args': ()}

The args and kwargs properties can be used to invoke functions:
```python
def test(a, *, b):
    ...

sig = signature(test)
ba = sig.bind(10, b=20)
test(*ba.args, **ba.kwargs)
```


**See also: ** PEP 362 - Function Signature Object. The detailed specification, implementation details and examples.(https://peps.python.org/pep-0362/)

## Function Signature options

In [2]:
[k.description for k in inspect._ParameterKind.__members__.values()]

['positional-only',
 'positional or keyword',
 'variadic positional',
 'keyword-only',
 'variadic keyword']

In [3]:
[k.name for k in inspect._ParameterKind.__members__.values()]

['POSITIONAL_ONLY',
 'POSITIONAL_OR_KEYWORD',
 'VAR_POSITIONAL',
 'KEYWORD_ONLY',
 'VAR_KEYWORD']

**Example:** 

In [4]:
def foo(a, /, b, *c, d, e=10, **f):
    pass

sig = signature(foo)


*Count the number of parameters of each type*

In [5]:
Counter(param.kind.description for param in sig.parameters.values())


Counter({'keyword-only': 2,
         'positional-only': 1,
         'positional or keyword': 1,
         'variadic positional': 1,
         'variadic keyword': 1})

In [6]:
Counter(param.kind.name for param in sig.parameters.values())


Counter({'KEYWORD_ONLY': 2,
         'POSITIONAL_ONLY': 1,
         'POSITIONAL_OR_KEYWORD': 1,
         'VAR_POSITIONAL': 1,
         'VAR_KEYWORD': 1})

*List all parameters, their type and their default (if it exists):*

In [7]:
for param in sig.parameters.values():
    if param.default is param.empty:
        dft = ''
    else:
        dft = param.default
    print(f'{param.name}\t{param.kind.description:25s}{dft}')


a	positional-only          
b	positional or keyword    
c	variadic positional      
d	keyword-only             
e	keyword-only             10
f	variadic keyword         


*print all keyword-only arguments without default values:*

In [8]:
for param in sig.parameters.values():
    if (param.kind == param.KEYWORD_ONLY and
                       param.default is param.empty):
        print('Parameter:', param)


Parameter: d


```python
def foo(a, /, b, *c, d, e=10, **f):
    pass
```

In [9]:
bnd = sig.bind('A', 'B', d='D')
bnd.apply_defaults()
print(bnd)
bnd.kwargs

<BoundArguments (a='A', b='B', c=(), d='D', e=10, f={})>


{'d': 'D', 'e': 10}

In [10]:
bnd.arguments

{'a': 'A', 'b': 'B', 'c': (), 'd': 'D', 'e': 10, 'f': {}}

In [ ]:
def print_sig_info(sig):
    for param in sig.parameters.values():
        if param.default is param.empty:
            dft = ''
        else:
            dft = param.default
        print(f'{param.name}\t{param.kind.description:25s}{dft}')

## Reuse of function signature, type hints and intellisense `*args` `**kwargs`

I was writing some python and got stumbled by an issue. I have tried to search for answers but seem to be lacking good keywords to find what I'm looking for.

I'm trying to reuse a function signature form another function. The end goal is to have intellisense and low maintenance of the wrapper.

```python
class B:
    """ Foreign class from other package """
    def __enter__(self):
        ...
    def __exit__(self, *_):
        ...
    def send(self, msg, timeout, ...):  # The function signature I want to re use for my function func. 
        ...
    def get(self):
        ...

class A:
    def __init__(self):
        self.other = B()  # I don't control this class

    def func(self, *args, **kwargs):  # <- I want to make MyPy and Intellisense to show hits from the B.send method. 
        with self.other as connection:
            connection.send(*args, **kwargs)  # fire and forget method
            response = connection.get()  # get data from a buffer (takes no arguments)
        return response
```

Is there a way to use the function signature from `self.other.send` as the function signature for `A.func`?

Searching for something like `functools.Wraps` but for this usage.

I've tried using `functools.Wraps`, looking into `ParamSpec`. Oh man typing takes a while to get into.

### Answer
Yes, you can do this by creating a no-op decorator factory. The construct looks like this:

```python
import typing as t

P = t.ParamSpec("P")
R = t.TypeVar("R")

def withSignatureFrom(
    f: t.Callable[t.Concatenate[t.Any, P], R], /
) -> t.Callable[
    [t.Callable[t.Concatenate[t.Any, P], R]], t.Callable[t.Concatenate[t.Any, P], R]
]:
    return lambda _: _
```

- `f: t.Callable[t.Concatenate[t.Any, P], R]` indicates the signature to use;
-> `t.Callable ...` is the decorator returned by the decorator factory. At runtime, this is the no-op `lambda _: _` (a decorator which just returns the decorated object unchanged).

`Concatenate[t.Any, P]` is to prevent type mismatches in the first (`self`) parameter, because in your case, you're using an instance method signature from `class B` on `class A`, which are unrelated. To clarify, if you wanted exact typing, the decorator factory may look like this instead:

```python
def withSignatureFrom(
    f: t.Callable[t.Concatenate["B", P], R], /
) -> t.Callable[
    [t.Callable[t.Concatenate["A", P], R]], t.Callable[t.Concatenate["A", P], R]
]:
    return lambda _: _
```

But exact typing of the instance parameter is rarely useful (and if you do this, you can't use this construct outside of the classes `B` and `A`).

Complete minimal working example:

In [12]:
import typing as t
import typing_extensions as tx
P = str
R = int

def withSignatureFrom(
    f: t.Callable[t.Concatenate[t.Any, P], R], /
) -> t.Callable[
    [t.Callable[t.Concatenate[t.Any, P], R]], t.Callable[t.Concatenate[t.Any, P], R]
]:
    return lambda _: _

class B:
    """Foreign class from other package"""
    def __enter__(self) -> tx.Self: ...
    def __exit__(self, *_: t.Any) -> None: ...
    def send(self, msg, timeout): ...
    def get(self): ...

class A:
    other: B

    def __init__(self) -> None:
        self.other = B()  # I don't control this class

    @withSignatureFrom(B.send)
    def func(self, *args, **kwargs):
        with self.other as connection:
            connection.send(*args, **kwargs)  # fire and forget method
            response = connection.get()  # get data from a buffer (takes no arguments)
        return response

TypeError: The last parameter to Concatenate should be a ParamSpec variable or ellipsis.